# Trabalho Prático 2 de Estruturas Criptográficas

- **Autores:** (Grupo 9)
     - Nelson Faria (A84727)
     - Miguel Oliveira (A83819)

## *Criptosistemas pós-quânticos PKE/KEM* - NTRU

Implementar o esquema NTRU em classes *Python/SageMath* apresentando, para cada um, as versões **KEM-IND-CPA** e **PKE-IND-CCA**.

Nota: Baseado no documento NTRU (https://ntru.org/f/ntru-20190330.pdf)

Conjuntos de parâmetros ntru-hrss específicos são denotados ntruhrss(n), por exemplo **ntruhrss701** é ntru-hrss com **n = 701**. O conjunto de parâmetros ntru-hrss recomendado é ntruhrss701, de acordo com o que foi submetido na 3ªronda do PQC.

### NTRU - KEM-IND-CPA



In [27]:
'''
Util para converter listas de inteiros para uma string que apresente esses inteiros por ordem correta
'''
def _toStr(intList):
    bin = BinaryStrings()
    s = [str(i) for i in intList]
    return bin.encoding("".join(s))
    

    
def _toBytes(salt):
    bts = [int(s).to_bytes( 1 , byteorder='big', signed=True ) for s in salt]
    res = b''.join(bts)
    return res

print(_toStr([1,2,5,7]))

print(_toBytes([1,2,5,7]))

#n = 701
#_Q.<x>  = Integers(3)[]
# Anel quociente Z[x]/(3,φn)
#phin = (x^n - 1)/(x-1)
#S3.<x>  = QuotientRing(_Q , _Q.ideal(phin))
#print(S3)
i = 5
N = 701
T = N // 4
u = [random.choice([-1,0,1]) for i in range(N)]
print(u)

00110001001100100011010100110111
b'\x01\x02\x05\x07'
[0, -1, 1, -1, 0, 0, 0, -1, 1, -1, 1, -1, -1, 0, -1, 0, -1, 0, 0, 0, 1, 0, 0, -1, 1, 1, 1, -1, -1, 0, -1, -1, 0, 1, 0, 0, 1, 1, 1, 0, 1, -1, 1, 0, 1, 0, 0, 0, 1, -1, 1, -1, -1, 1, 0, 1, -1, -1, 1, 0, 0, 1, -1, 0, 0, 0, 0, -1, 1, 0, -1, 1, 0, -1, 1, 0, 1, -1, 0, 1, 0, -1, 1, 0, 0, 0, -1, -1, -1, 0, -1, -1, 1, 1, 1, 1, 0, -1, 1, 1, 1, 1, 1, -1, 1, 0, 1, -1, 1, 0, 1, 0, 0, 1, -1, -1, -1, -1, 1, 0, -1, 0, 0, -1, 0, 1, -1, 1, 1, 1, 0, 0, -1, -1, -1, 1, -1, 0, 1, 1, -1, 1, 0, 1, 0, -1, -1, 1, 1, 0, 0, -1, 0, 1, -1, -1, -1, 1, 0, 1, -1, -1, 0, 1, 1, 1, -1, 0, 0, 0, 0, 0, -1, 0, 1, -1, -1, -1, 1, 0, 1, -1, 0, 0, 0, 0, 0, 0, -1, 1, -1, 0, 1, 0, 0, -1, -1, 1, 0, -1, 1, 0, -1, 0, 0, 1, 1, -1, 0, 0, -1, -1, 0, -1, -1, 1, 0, 1, 1, 1, 0, 0, -1, 1, 0, -1, -1, 0, 1, 0, -1, 0, -1, 1, 1, 0, -1, 0, 0, -1, -1, 1, 0, -1, 0, -1, -1, 1, 0, -1, -1, 0, 0, -1, -1, 1, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 1, -1, -1, 1, -1, 0, -1, 1, 0, 0, 1, 1, 1, -1, 1, 1, 1, -1, 0,

In [35]:
import random, os, hashlib

n = 701

# Converte uma sequencia de inteiros na sua representacao binaria
def randomBitString(size):
    
    # Gera uma sequencia de n bits aleatorios
    u = [random.choice([0,1]) for i in range(size)]
    # Mistura os valores da lista, so para aumentar a aleatoriedade
    random.shuffle(u)
    return u


def Ternary(bit_string):
    
    # S3 -> Anel quociente Z[x]/(3,φn)
    _Q.<x>  = Integers(3)[]
    S3.<x>  = QuotientRing(_Q , _Q.ideal((x^n - 1)/(x-1)))
    v = 0
    for i in range(n-1):
        
        v = v + sum([2^j*bit_string[8*i+j+1] for j in range(7)])*x^i
    
    return S3(v)
        

def Ternary_Plus(bit_string):
    
    # S3 -> Anel quociente Z[x]/(3,φn)
    _Q.<x>  = Integers(3)[]
    S3.<x>  = QuotientRing(_Q , _Q.ideal((x^n - 1)/(x-1)))
    
    # Set v = Ternary((b1; b2; ... ; bl))
    v = list(Ternary(bit_string))
    # t = somatorio de 0 ate n-2 de vi*vi+1
    t = sum([v[i]*v[i+1] for i in range(n-2)])
    # Set s = -1 if t < 0, otherwise set s = 1
    if t<0:
        s = -1
    else:
        s = 1
    # Set i = 0
    i = 0
    while i<n-1:
        # Set vi = s . vi
        v[i] = s*v[i]
        # Set i = i + 2
        i = i + 2
    
    return S3(v)
    

def Sample_fg(seed):
    
    # sample_iid_bits = 8*n - 8
    sample_iid_bits = 8*n - 8
    
    # Parse de fg_bits em f_bits||g_bits
    f_bits = seed[:sample_iid_bits]
    g_bits = seed[sample_iid_bits:]
    
    # Definir f = Ternary_Plus(f_bits)
    f = Ternary_Plus(f_bits)
    # Definir g0 = Ternary_Plus(g_bits)
    g0 = Ternary_Plus(g_bits)
    # Definir g = φ1.g0
    g = g0 * (x-1)
    
    return (f,g)


def Sample_rm(coins):
    
    # sample_iid_bits = 8*n - 8
    sample_iid_bits = 8*n - 8
    
    # Parse de rm_bits em r_bits||m_bits
    r_bits = coins[:sample_iid_bits]
    m_bits = coins[sample_iid_bits:]
    
    # Set r = Ternary(r_bits)
    r = Ternary(r_bits)
    # Set m = Ternary(m_bits)
    m = Ternary(m_bits)
    
    return (r,m)
    
    
# Precisamos de ter uma seed com 11200 bits, ou seja, 1400 bytes
Sample_fg(randomBitString(11200))
#Sample_rm(randomBitString(11200))
#print(Sample_fg(randomBitString(11200)))

NameError: name 'S3' is not defined

In [49]:
_Q.<x>  = Integers(8192)[]
Sq.<x> = QuotientRing(_Q , _Q.ideal((x^791 - 1)//(x-1)))

def _toZ(f,p=None):
    ff = list(f)
    if p == None:
        return ff
    else:
        fp = map(lift,[Mod(a,p) for a in ff])
        return [u if u <= p//2 else u-p for u in fp ]
    
a = Sq(2*x^697 + x^695 + x^692 + x^690 + 2*x^689 + x^688 + 2*x^686 + 2*x^685 + x^683 + x^681 + 2*x^675 + x^673 + 2*x^672 + x^670 + 2*x^666 + 2*x^665 + 2*x^664 + 2*x^663 + 2*x^662 + 2*x^660 + 2*x^659 + 2*x^657 + 2*x^656 + x^655 + x^653 + 2*x^652 + x^650 + 2*x^648 + x^646 + 2*x^644 + x^643 + 2*x^641 + x^640 + x^639 + 2*x^637 + x^636 + 2*x^635 + x^634 + 2*x^632 + x^631 + x^630 + x^628 + 2*x^626 + 2*x^625 + 2*x^624 + x^621 + 2*x^620 + 2*x^619 + 2*x^618 + x^616 + x^613 + 2*x^612 + x^611 + x^610 + 2*x^607 + 2*x^606 + 2*x^604 + 2*x^603 + x^601 + 2*x^599 + x^598 + 2*x^596 + x^595 + 2*x^593 + x^592 + 2*x^591 + x^590 + 2*x^589 + x^585 + x^584 + x^582 + x^581 + x^580 + x^579 + 2*x^578 + 2*x^577 + 2*x^572 + 2*x^571 + x^570 + x^567 + 2*x^566 + 2*x^565 + x^564 + x^562 + x^561 + 2*x^558 + 2*x^555 + 2*x^553 + 2*x^552 + 2*x^551 + 2*x^550 + x^549 + 2*x^548 + x^547 + x^546 + 2*x^545 + 2*x^544 + x^542 + 2*x^541 + x^539 + 2*x^538 + x^535 + 2*x^534 + x^532 + 2*x^530 + 2*x^527 + 2*x^525 + 2*x^524 + x^523 + x^516 + x^512 + x^509 + x^508 + x^507 + x^506 + 2*x^505 + 2*x^503 + x^502 + 2*x^501 + 2*x^500 + 2*x^497 + x^495 + x^493 + x^492 + x^491 + 2*x^489 + 2*x^488 + 2*x^486 + x^484 + 2*x^483 + x^482 + x^481 + x^480 + x^478 + x^477 + 2*x^476 + 2*x^475 + x^473 + x^470 + 2*x^469 + 2*x^468 + x^467 + x^466 + x^464 + x^462 + 2*x^461 + 2*x^460 + x^457 + x^456 + 2*x^453 + x^452 + 2*x^451 + 2*x^448 + x^447 + 2*x^446 + 2*x^445 + x^444 + 2*x^443 + 2*x^442 + x^439 + x^438 + x^437 + x^436 + 2*x^435 + x^434 + 2*x^433 + 2*x^432 + x^430 + 2*x^428 + x^427 + 2*x^426 + 2*x^425 + 2*x^424 + x^422 + x^421 + 2*x^418 + 2*x^416 + 2*x^415 + 2*x^413 + 2*x^412 + x^411 + 2*x^410 + x^407 + x^406 + 2*x^405 + 2*x^404 + 2*x^403 + 2*x^401 + 2*x^400 + 2*x^399 + 2*x^398 + 2*x^396 + x^395 + x^394 + x^393 + x^391 + x^389 + x^388 + 2*x^386 + 2*x^385 + 2*x^384 + x^383 + 2*x^382 + 2*x^381 + x^379 + 2*x^377 + x^374 + x^372 + x^371 + 2*x^370 + x^369 + x^368 + 2*x^367 + 2*x^366 + 2*x^363 + x^362 + x^360 + 2*x^359 + 2*x^358 + 2*x^357 + 2*x^356 + 2*x^355 + x^353 + 2*x^352 + 2*x^351 + 2*x^350 + x^349 + x^348 + 2*x^346 + x^344 + x^343 + x^342 + x^340 + 2*x^339 + 2*x^338 + x^337 + x^333 + 2*x^332 + x^331 + x^330 + 2*x^327 + x^326 + x^325 + 2*x^324 + x^322 + 2*x^321 + x^320 + x^318 + 2*x^317 + x^315 + 2*x^314 + 2*x^312 + x^311 + 2*x^310 + 2*x^309 + 2*x^308 + x^307 + x^306 + x^305 + x^304 + x^303 + 2*x^302 + 2*x^301 + x^299 + x^298 + 2*x^296 + x^293 + x^292 + 2*x^289 + x^288 + 2*x^286 + x^285 + x^284 + 2*x^283 + x^282 + x^281 + x^279 + 2*x^277 + x^275 + x^274 + x^272 + 2*x^271 + 2*x^270 + 2*x^268 + 2*x^267 + x^266 + x^262 + 2*x^261 + x^258 + 2*x^257 + x^256 + x^251 + 2*x^249 + x^248 + 2*x^245 + x^244 + 2*x^243 + x^242 + x^240 + x^239 + 2*x^238 + 2*x^234 + 2*x^233 + x^232 + x^230 + x^229 + x^228 + x^226 + 2*x^225 + x^224 + 2*x^222 + 2*x^221 + 2*x^219 + x^218 + x^214 + 2*x^212 + 2*x^208 + 2*x^207 + 2*x^206 + x^205 + 2*x^204 + 2*x^203 + x^202 + x^201 + 2*x^200 + 2*x^199 + 2*x^198 + 2*x^197 + 2*x^196 + x^195 + 2*x^194 + 2*x^192 + 2*x^190 + 2*x^189 + x^188 + x^187 + x^186 + 2*x^184 + x^183 + 2*x^180 + x^179 + 2*x^178 + x^177 + 2*x^176 + 2*x^175 + x^173 + 2*x^171 + 2*x^170 + x^169 + 2*x^168 + 2*x^166 + 2*x^165 + 2*x^163 + 2*x^159 + 2*x^158 + 2*x^157 + 2*x^156 + 2*x^155 + x^154 + 2*x^148 + x^147 + x^146 + x^145 + x^144 + x^143 + x^142 + x^141 + 2*x^140 + 2*x^139 + x^135 + x^133 + 2*x^132 + x^130 + 2*x^129 + x^127 + 2*x^125 + x^123 + 2*x^120 + x^118 + 2*x^117 + 2*x^116 + 2*x^115 + x^109 + x^108 + x^107 + x^103 + x^99 + x^94 + 2*x^92 + x^91 + x^90 + x^89 + 2*x^85 + x^83 + x^82 + x^81 + x^80 + 2*x^79 + 2*x^76 + 2*x^75 + 2*x^74 + x^73 + x^69 + 2*x^68 + 2*x^66 + x^65 + x^64 + 2*x^61 + x^60 + x^58 + 2*x^57 + x^56 + 2*x^53 + 2*x^52 + x^50 + 2*x^49 + x^48 + 2*x^47 + x^45 + 2*x^44 + 2*x^43 + x^42 + x^41 + 2*x^39 + x^38 + 2*x^36 + x^35 + 2*x^34 + 2*x^33 + 2*x^32 + 2*x^31 + x^27 + x^25 + x^24 + 2*x^23 + x^22 + 2*x^21 + 2*x^20 + 2*x^18 + x^17 + x^16 + 2*x^15 + x^14 + x^12 + x^11 + 2*x^10 + 2*x^9 + 2*x^7 + x^6 + x^5 + 2*x^4 + 2*x^3 + 1)

print(_toZ(a) == list(a))

#print(a.inverse_of_unit())

# S3 -> Anel quociente Z[x]/(3,φn)
_Q.<x>  = Integers(3)[]
S3.<x>  = QuotientRing(_Q , _Q.ideal((x^701 - 1)/(x-1)))
b = [i-1 for i in list(a)]
print(b)

print(S3(-1))

True
[0, 8191, 8191, 1, 1, 0, 0, 1, 8191, 1, 1, 0, 0, 8191, 0, 1, 0, 0, 1, 8191, 1, 1, 0, 1, 0, 0, 8191, 0, 8191, 8191, 8191, 1, 1, 1, 1, 0, 1, 8191, 0, 1, 8191, 0, 0, 1, 1, 0, 8191, 1, 0, 1, 0, 8191, 1, 1, 8191, 8191, 0, 1, 0, 8191, 0, 1, 8191, 8191, 0, 0, 1, 8191, 1, 0, 8191, 8191, 8191, 0, 1, 1, 1, 8191, 8191, 1, 0, 0, 0, 0, 8191, 1, 8191, 8191, 8191, 0, 0, 0, 1, 8191, 0, 8191, 8191, 8191, 8191, 0, 8191, 8191, 8191, 0, 8191, 8191, 8191, 0, 0, 0, 8191, 8191, 8191, 8191, 8191, 1, 1, 1, 0, 8191, 1, 8191, 8191, 0, 8191, 1, 8191, 0, 8191, 1, 0, 8191, 1, 0, 8191, 0, 8191, 8191, 8191, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 8191, 8191, 8191, 8191, 8191, 0, 1, 1, 1, 1, 1, 8191, 8191, 8191, 1, 8191, 1, 1, 8191, 1, 0, 1, 1, 8191, 0, 8191, 1, 1, 0, 1, 0, 1, 8191, 8191, 0, 1, 8191, 0, 0, 0, 1, 1, 8191, 1, 8191, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 8191, 8191, 8191, 1, 8191, 0, 8191, 8191, 8191, 0, 1, 8191, 1, 1, 8191, 0, 1, 0, 8191, 0, 0, 0, 8191, 0, 1, 1, 8191, 8191, 8191, 1, 0, 0, 8191, 0, 1, 0

In [148]:
import hashlib

m = hashlib.sha3_256()
m.update(b"Nelson Faria")
m.digest()
#m.digest_size

b"\xce\xef\x01\x92%\xb4\x1d\xc4g\x9ca\xef\x02*\xa1<\x8a\x96Q\xf1B\xe7h\xa7Q\xa4'\xb4\xb9\xb9&\xf8"

In [45]:
import random, hashlib
import numpy as np

# Baseado no esquema da página 25 do documento https://ntru.org/f/ntru-20190330.pdf

class NTRU_KEM(object):
    
    def __init__(self, N=701, Q=8192, timeout=None):
        
        # Todas as inicializações de parâmetros são baseadas na submissao com os parâmetros do NTRUHRSS-701, onde n = 701
        self.n = N
        self.q = Q
        
        # Definição dos aneis
        # R -> Anel quociente Z[x]/(φ1.φn) -> Em principio , nao necessario
        #_Z.<x>  = ZZ[]
        #R.<x> = QuotientRing(_Z , _Z.ideal(x^self.n - 1))
        #self.R = R
        ## Rq -> Anel quociente Z[x]/(q,φ1.φn)
        #_Q.<x>  = Integers(self.q)[]
        #Rq.<x> = QuotientRing(_Q , _Q.ideal(x^self.n - 1))
        #self.Rq = Rq
        ## Sq -> Anel quociente Z[x]/(q,φn)
        #_Q.<x>  = Integers(self.q)[]
        #Sq.<x> = QuotientRing(_Q , _Q.ideal((x^self.n - 1)//(x-1)))
        #self.Sq = Sq
        ## S3 -> Anel quociente Z[x]/(3,φn)
        #_Q.<x>  = Integers(3)[]
        #S3.<x> = QuotientRing(_Q , _Q.ideal((x^self.n - 1)//(x-1)))
        #self.S3 = S3
        # Outra versão
        Zx.<x>  = ZZ[]
        Qq1 = PolynomialRing(GF(self.q), 'x')
        Qq2  = PolynomialRing(GF(self.q), 'x')
        Q3  = PolynomialRing(GF(3), 'x')
        # Definicoes principais
        x = Zx.gen()
        y = Qq1.gen()
        w = Qq2.gen()
        z = Q3.gen()
        R = QuotientRing(Zx, x^self.n-1)
        self.R = R
        Rq = QuotientRing(Qq1, y^self.n-1)
        self.Rq = Rq
        Sq = QuotientRing(Qq2, ((w^self.n - 1)//(w-1)))
        self.Sq = Sq
        S3 = QuotientRing(Q3, ((z^self.n - 1)//(z-1)))
        self.S3 = S3
        
          
    # Converte uma sequencia de inteiros na sua representacao binaria
    def randomBitString(self, size):
        
        # Gera uma sequencia de n bits aleatorios
        u = [random.choice([0,1]) for i in range(size)]
        # Mistura os valores da lista, so para aumentar a aleatoriedade
        random.shuffle(u)
        return u
    
    
    # Gera um polinomio ternario 
    def Ternary(self, bit_string):
        
        x = self.S3.gen()
        v = 0
        for i in range(self.n-1):
            
            v = v + sum([2^j*bit_string[8*i+j+1] for j in range(7)])*x^i
        
        # Passar de valores inteiros {0,1,2} para {-1,0,1}
        return self.R([-1 if i==2 else i for i in list(self.S3(v))])
        #return self.S3(v)
    
    
    # Gera um polinomio ternario que satisfaz a non-negative correlation property
    def Ternary_Plus(self, bit_string):
        
        x = self.S3.gen()
        # Set v = Ternary((b1; b2; ... ; bl))
        v = list(self.Ternary(bit_string))
        # t = somatorio de 0 ate n-2 de vi*vi+1
        t = sum([v[i]*v[i+1] for i in range(self.n-2)])
        # Set s = -1 if t < 0, otherwise set s = 1
        if t < 0:
            s = -1
        else:
            s = 1
            
        # Set i = 0
        i = 0
        while i<self.n-1:
            # Set vi = s . vi
            v[i] = s*v[i]
            # Set i = i + 2
            i = i + 2
        
        # Passar de valores inteiros {0,1,2} para {-1,0,1}
        return self.R(v)
        #return self.S3(v)
    
    
    # Gera um polinomio f em Lf (no nosso caso, em T+) e um polinomio g em Lg (no nosso caso, em {φ1.v : v € T+})
    def Sample_fg(self, seed):
        
        x = self.R.gen()
        w = self.S3.gen()
        # sample_iid_bits = 8*n - 8
        sample_iid_bits = 8*self.n - 8
        
        # Parse de fg_bits em f_bits||g_bits
        f_bits = seed[:sample_iid_bits]
        g_bits = seed[sample_iid_bits:]
        
        # Definir f = Ternary_Plus(f_bits)
        f = self.Ternary_Plus(f_bits)
        # Definir g0 = Ternary_Plus(g_bits)
        g0 = self.Ternary_Plus(g_bits)
        # Definir g = φ1.g0
        g = g0 * (x-1)
        
        return (f,g)
    
    
    # Gera um polinomio r em Lr (no nosso caso, em T) e um polinomio m em Lm (no nosso caso, em T)
    def Sample_rm(self, coins):
        
        # sample_iid_bits = 8*n - 8
        sample_iid_bits = 8*self.n - 8
        
        # Parse de rm_bits em r_bits||m_bits
        r_bits = coins[:sample_iid_bits]
        m_bits = coins[sample_iid_bits:]
        
        # Set r = Ternary(r_bits)
        r = list(self.Ternary(r_bits))
        # Set m = Ternary(m_bits)
        m = list(self.Ternary(m_bits))
        
        #return (self.Rq(list(self.R(list(r)))),self.Rq(list(self.R(list(m)))))
        return (r,m)
    
    
    # Funcao usada para gerar o par de chaves pública e privada
    def geraChaves(self, seed):
        
        (f,g) = self.Sample_fg(seed)
        # fq <- (1/f) mod (q;φn)
        f1 = self.Sq(list(self.R(list(f))))
        fq = f1^(-1)
        # h <- (3.g.fq) mod(q;φ1φn)
        #g1 = self.Rq(list(self.R(list(g))))
        #fq1 = self.Rq(list(fq))
        h = self.Rq(3*g*fq)
        # hq <- (1/h) mod(q;φn)
        h1 = self.Sq(list(h))
        hq = h1^(-1)
        # fq <- (1/f) mod(3;φn)
        f2 = self.S3(list(f))
        fq = f2^(-1)
        
        return {'sk' : (f,fq,hq) , 'pk' : h}

    
    # Recebe como parametros a chave publica e o tuplo (r,m)
    def cifra(self, h, rm):
        
        w = self.S3.gen()
        y = self.Rq.gen()
        # m' <- Lift(m), de acordo com a variante HRSS: Lift(m) = φ1 . S3(m/φ1)
        liftm = rm[1]
        # c <- (r.h + m') mod (q,φ1φn)
        c = self.Rq(rm[0])*h + self.Rq(liftm)
        return c
    
    
    # Recebe como parametros a chave privada (f,fp,hq) e ainda o criptograma
    def decifra(self, sk, c):
        
        w = self.S3.gen()
        # 
        Q.<y>  = PolynomialRing(GF(self.q))
        P.<y> = Q.quotient(y-1)
        print(P(list(c)))
        if P(list(c)) != 0:
            return (0,0,1)
        
        # a <- (c.f) mod(q,φ1φn)
        a = c*self.Rq(list(self.R(list(sk[0]))))
        # m <- (a.fp) mod(3,φn)
        m = self.S3(list(a)) * sk[1]
        # m' <- Lift(m), de acordo com a variante HRSS: Lift(m) = φ1 . S3(m/φ1)
        liftm = m
        # r <- ((c-m').hq) mod(q,φn)
        r = self.Sq(list(c-self.Rq(list(self.R(list(liftm))))))*sk[2]
        
        return (r,m,0)

        
    
    # Funcao que serve para encapsular a chave que for acordada a partir de uma chave publica
    def encapsula(self, e, n):
        
        return True
    
    # Funcao usada para desencapsular uma chave, a partir do seu "encapsulamento"
    def desencapsula(self, cs):
        
        return True
        

In [50]:
ntru = NTRU_KEM()

keys = ntru.geraChaves(ntru.randomBitString(11200))

rm = ntru.Sample_rm(ntru.randomBitString(11200))

c = ntru.cifra(keys['pk'], (rm[0],rm[1]))

rmDec = ntru.decifra(keys['sk'], c)

Q3  = PolynomialRing(GF(3), 'x')
z = Q3.gen()
S3 = QuotientRing(Q3, ((z^701 - 1)//(z-1)))

print(rmDec[1])
print(S3(rm[1]))
print(rmDec[1] == S3(rm[1]))

1
0
2*xbar^698 + 2*xbar^697 + 2*xbar^696 + 2*xbar^695 + xbar^694 + xbar^692 + 2*xbar^689 + xbar^688 + xbar^686 + 2*xbar^685 + 2*xbar^684 + xbar^682 + 2*xbar^680 + xbar^677 + 2*xbar^676 + xbar^675 + 2*xbar^674 + xbar^671 + 2*xbar^670 + xbar^669 + xbar^668 + 2*xbar^667 + xbar^665 + xbar^663 + 2*xbar^662 + 2*xbar^660 + xbar^659 + 2*xbar^658 + xbar^656 + 2*xbar^655 + xbar^654 + 2*xbar^653 + 2*xbar^652 + 2*xbar^651 + 2*xbar^650 + xbar^649 + xbar^648 + xbar^644 + xbar^643 + xbar^639 + 2*xbar^638 + xbar^637 + xbar^635 + 2*xbar^633 + 2*xbar^632 + 2*xbar^630 + 2*xbar^628 + xbar^626 + xbar^625 + xbar^624 + 2*xbar^623 + 2*xbar^621 + 2*xbar^620 + xbar^619 + xbar^617 + xbar^616 + xbar^615 + 2*xbar^614 + xbar^613 + xbar^612 + xbar^611 + 2*xbar^609 + xbar^608 + xbar^606 + 2*xbar^602 + 2*xbar^599 + 2*xbar^597 + xbar^596 + xbar^594 + xbar^593 + 2*xbar^592 + 2*xbar^591 + 2*xbar^589 + 2*xbar^587 + 2*xbar^586 + xbar^584 + xbar^583 + 2*xbar^580 + 2*xbar^579 + 2*xbar^577 + xbar^576 + 2*xbar^575 + xbar^574 +

In [194]:
# VER ISTO!!!
m = hashlib.sha3_256()
m.update(b"Nelson Faria")
representacaoBinaria = bin(int(m.hexdigest(), base=16))[2:].zfill(8)
print(representacaoBinaria)

representacaoHex = hex(int(representacaoBinaria, base=2))

print(m.hexdigest())
print(representacaoHex)
print('0x' + m.hexdigest() == representacaoHex)


1100111011101111000000011001001000100101101101000001110111000100011001111001110001100001111011110000001000101010101000010011110010001010100101100101000111110001010000101110011101101000101001110101000110100100001001111011010010111001101110010010011011111000
ceef019225b41dc4679c61ef022aa13c8a9651f142e768a751a427b4b9b926f8
0xceef019225b41dc4679c61ef022aa13c8a9651f142e768a751a427b4b9b926f8
True


In [3]:
F = PolynomialRing(GF(32),'a')
a = F.gen()
Ring = F.quotient(a^11 - 1, 'x')
x = Ring.gen()
pollist = [-1, 1, 1, 0, -1, 0, 1, 0, 0, 1, -1]

fq = Ring(pollist)
print(fq)
print(fq^(-1))

x^10 + x^9 + x^6 + x^4 + x^2 + x + 1
x^5 + x + 1
